<a href="https://colab.research.google.com/github/habibarezq/ML-Assignments-25/blob/main/Assignment-2/notebooks/cnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [2]:
# comment if you are on vs code
from google.colab import files
uploaded = files.upload()

from data_preprocessing import *
from nn_manual import *

Saving data_preprocessing.py to data_preprocessing.py
Saving nn_manual.py to nn_manual.py


# 1) MNIST Dataset & Dataloaders


In [3]:
multi_data_cnn = MNISTDataLoader(batch_size=64, binary=False,flatten=False)
train_loader_cnn, val_loader_cnn, test_loader_cnn = multi_data_cnn.get_loaders()

multi_data = MNISTDataLoader(batch_size=64, binary=False)
train_loader, val_loader, test_loader = multi_data.get_loaders()

100%|██████████| 9.91M/9.91M [00:00<00:00, 131MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 21.8MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 102MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.26MB/s]


# 3) Simple CNN


In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)   # 1→16 filters, 3x3
        self.pool  = nn.MaxPool2d(2, 2)
        self.fc    = nn.Linear(16 * 13 * 13, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # conv → relu → pool
        x = x.view(x.size(0), -1)              # flatten
        return self.fc(x)

# 4) Training Function

In [5]:
def train_model(model, loader, epochs=1):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    loss_fn = nn.CrossEntropyLoss()
    for _ in range(epochs):
        for imgs, labels in loader:
            preds = model(imgs)
            loss = loss_fn(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


# 5) Test Accuracy

In [9]:
def test_accuracy(model,test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            preds = model(imgs)
            correct += (preds.argmax(1) == labels).sum().item()
            total += labels.size(0)
    return correct / total

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

fc = CustomFeedforwardNN()
cnn = SimpleCNN()

print("Training FC Model...")
train_model_once(fc, train_loader,val_loader, epochs=10,device=device)
print("Training CNN Model...")
train_model(cnn, train_loader_cnn, epochs=10)

fc_acc = test_accuracy(fc,test_loader)
cnn_acc = test_accuracy(cnn,test_loader_cnn)

print("\n=== RESULTS ===")
print(f"Fully Connected Accuracy: {fc_acc*100:.2f}%")
print(f"CNN Accuracy:            {cnn_acc*100:.2f}%")
print("\nCNN performs better because it learns SPATIAL features (edges, shapes) from the image instead of treating it as flat pixels.")


Training FC Model...
Epoch 1/10 Train Loss: 1.0991 Train Acc: 0.7215 Val Loss: 0.5270 Val Acc: 0.8607
Epoch 2/10 Train Loss: 0.4302 Train Acc: 0.8845 Val Loss: 0.3742 Val Acc: 0.8962
Epoch 3/10 Train Loss: 0.3454 Train Acc: 0.9024 Val Loss: 0.3217 Val Acc: 0.9117
Epoch 4/10 Train Loss: 0.3080 Train Acc: 0.9131 Val Loss: 0.2990 Val Acc: 0.9160
Epoch 5/10 Train Loss: 0.2827 Train Acc: 0.9187 Val Loss: 0.2793 Val Acc: 0.9211
Epoch 6/10 Train Loss: 0.2625 Train Acc: 0.9251 Val Loss: 0.2613 Val Acc: 0.9274
Epoch 7/10 Train Loss: 0.2458 Train Acc: 0.9295 Val Loss: 0.2490 Val Acc: 0.9321
Epoch 8/10 Train Loss: 0.2315 Train Acc: 0.9348 Val Loss: 0.2333 Val Acc: 0.9348
Epoch 9/10 Train Loss: 0.2187 Train Acc: 0.9377 Val Loss: 0.2278 Val Acc: 0.9371
Epoch 10/10 Train Loss: 0.2076 Train Acc: 0.9410 Val Loss: 0.2138 Val Acc: 0.9414
Training CNN Model...

=== RESULTS ===
Fully Connected Accuracy: 93.54%
CNN Accuracy:            93.57%

CNN performs better because it learns SPATIAL features (edges, 